<a href="https://colab.research.google.com/github/Scott-S-Lin/NTUT_PhD/blob/main/Lab7_scott.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorBoard進階技巧

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/taipeitechmmslab/MMSLAB-TF2/blob/master/Lab7.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/taipeitechmmslab/MMSLAB-TF2/blob/master/Lab7.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## tf.summary

tf.summary是TensorFlow提供TensorBoard低階API指令，主要是用來紀錄log檔。
以下整理了幾個常用功能：
- tf.scalar：儲存顯示如損失、指標或學習率等的變化趨勢。
- tf.image：儲存顯示影像。
- tf.audio：儲存顯示可播放的音頻。
- tf.histogram：儲存顯示模型權重。
- tf.text：儲存顯示一段文字。


In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

### 創建TensorBoard log檔

In [ ]:
summary_writer = tf.summary.create_file_writer('lab7-logs-summary')

### Scalar

In [ ]:
# 在0~2π之間產生100個點
x = np.linspace(0, 2 * np.pi , 100)
# 將100個點帶入sin函數中
data = np.sin(x)
with summary_writer.as_default():  # summary_writer作為預設寫入的紀錄檔
    for i, y in enumerate(data):
        tf.summary.scalar('sin', y, step=i)  # 存入數值(y為數值，i為時間軸)

### Image

1. 儲存一張影像在紀錄檔中並顯示

In [ ]:
# 建立讀取影像的函數
def read_img(file):
    image_string = tf.io.read_file(file)  # 讀取檔案
    # 將讀入檔案以影像格式來解碼
    image_decode = tf.image.decode_image(image_string)
    # 將影像增加一個維度(number,height,width,channel)方便之後存入紀錄檔中
    # image_decode = tf.expand_dims(image_decode, axis=0)
    return image_decode

img = read_img('image/airplane.png')  # 讀入影像資訊
plt.imshow(img)  # 顯示讀入的影像資訊

In [ ]:
image_string = tf.io.read_file('image/airplane.png')  # 讀取檔案
image_decode = tf.image.decode_image(image_string)

In [ ]:
with summary_writer.as_default():  # summary_writer作為預設寫入的紀錄檔
    tf.summary.image("Airplane", [image_decode], step=0)  # 存入影像資訊

2. 一次儲存五張影像到紀錄檔中並顯示

In [ ]:
!ls image/

In [ ]:
img_files = ['airplane_zoom.png', 'airplane_flip.png', 'airplane_color.png', 'airplane_rot.png',
             'airplane.png']  # 創建一個陣列用來儲存讀入的影像 

imgs = [] 

for file in img_files: 
    imgs.append(read_img('image/'+file))  # 讀取影像並存入陣列中 

with summary_writer.as_default():  # summary_writer作為預設寫入的紀錄檔 
    # 一次存入五張影像(注意:如果max_outputs沒設定為5，就只會儲存3張影像) 
    tf.summary.image("Airplane Augmentation", imgs, max_outputs=5, step=0) 

3. 將五張影像以不同Step(時間)儲存

In [ ]:
with summary_writer.as_default():  # summary_writer作為預設寫入的紀錄檔
    # 每次儲存一張影像，並儲存在不同Step中
    for i, img in enumerate(imgs):
        tf.summary.image("Save image each step", [img], step=i)

### Text

In [ ]:
# 建立一個陣列，裡面包含了對話記錄
texts = ["小明：Cubee小助理最近好想學深度學習的技術哦!", 
         "Cubee：這是當然的阿，這可現今最火的技術呢!", 
         "小明：那我該如何入門呢?", 
         "Cubee：推薦你一本書「輕鬆學會Google TensorFlow2.0深度學習」。", 
         "小明：這本書沒有深度學習經驗的人也能學會嗎?", 
         "Cubee：這是當然的，你只需要基礎Python能力就可以學會了!", 
         "小明：太好了那我要趕快去買了!"]

with summary_writer.as_default():  # summary_writer作為預設寫入的紀錄檔
    # 將每一段字串資訊以不同Step存入到記錄檔中
    for i, text in enumerate(texts):
        tf.summary.text("Chat record", text, step=i)

### Audio

In [ ]:
# 建立讀取音訊的函數
def read_audio(file):
    audio_string = tf.io.read_file(file)  # 讀取檔案
    # 將讀入檔案以音訊格式來解碼
    audio, fs = tf.audio.decode_wav(audio_string)
    # 因為tf.summary.audio要求輸入格式為[k(clips), t(frames), c(channels)]
    # 而解碼後的音訊只有[t(frames), c(channels)]，所以需要增加一個維度給音訊
    audio = tf.expand_dims(audio, axis=0)
    return audio, fs

audio, fs = read_audio('./audio/cat.wav')  # 讀取音訊檔

with summary_writer.as_default():  # summary_writer作為預設寫入的紀錄檔
    tf.summary.audio('cat', audio, fs, step=0)  # 存入音訊資訊

### Histogram

目前 TensorFlow-gpu 2.0-alpha有問題，等待下一版修復。

https://github.com/tensorflow/tensorboard/issues/1993

建立一個常態分佈

In [ ]:
data = tf.random.normal([64, 100], dtype=tf.float64)

儲存常態分佈分佈

In [ ]:
with summary_writer.as_default():
    tf.summary.histogram('Normal distribution', data, step=0)

儲存多個常態分佈，並且各個之間平均值都相差0.01

In [ ]:
with summary_writer.as_default():
    for i, offset in enumerate(tf.range(0, 10, delta=0.1, dtype=tf.float64)):
        tf.summary.histogram('Normal distribution 2', data+offset, step=i)

In [ ]:
%load_ext tensorboard

In [ ]:
tensorboard --logdir lab7-logs-summary/

# 實驗一：使用tf.summary.image紀錄訓練結果

### Import 必要套件

In [ ]:
import os
import io
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from preprocessing import parse_aug_fn, parse_fn

### Confusion matrix

Confusion matrix函數：透過tf.math.confusion_matrix來計算Confusion matrix。

In [ ]:
y_true = [2, 1, 0, 2, 2, 0, 1, 1]
y_pred = [0, 1, 0, 2, 2, 0, 2, 1]
cm = tf.math.confusion_matrix(y_true, y_pred, num_classes=3).numpy()
print(cm)

建立plot_confusion_matrix函式：將剛剛上方計算的Confusion matrix陣列以Matplotlib圖片來表示，而Confusion matrix中的數字改成百分比型式。

In [ ]:
def plot_confusion_matrix(cm, class_names):
    """
    產生一張圖表示的Confusion matrix
    
    Args:
    cm (shape = [n, n]): 傳入Confusion matrix
    class_names (shape = [n]): 傳入類別
    """
    # 標準化confusion matrix.
    cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)
    
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    tick_index = np.arange(len(class_names))
    # matplotlib 3.1.1 bug，如果不設定ylim在[-0.5~2.5]，圖片y軸範圍會被縮小成[0~2]
    plt.ylim([-0.5, 2.5])
    # Y軸顯示類別名稱
    plt.yticks(tick_index, class_names)
    # X軸顯示類別名稱，並將類別名稱旋轉45度(避免文字重疊)
    plt.xticks(tick_index, class_names, rotation=45)
    # 再圖片右邊產生一條顏色刻度條
    plt.colorbar()

    # 在每一格Confusion matrix輸入預測百分比
    threshold = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            # 如果格內背景顏色太深使用白色文字顯示，反之使用黑色文字
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)
            
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    # 將圖片的位置進行調整，避免x或y軸的文字被遮擋
    plt.tight_layout()
    return figure

# Example
img = plot_confusion_matrix(cm, [0, 1, 2])

建立plot_to_image函數：將Matplotlib圖片轉TensorFlow型式的圖片，這樣才能夠透過tf.summary.image紀錄影像到TensorBoard。

In [ ]:
def plot_to_image(figure):
    """將Matplotlib plot的圖片轉TensorFlow的張量格式"""
    # 將Matplotlib plot的圖片以PNG的格式儲存到記憶體中
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    # 關閉plt圖片，防止圖片直接顯示在Jupyter notebook介面中
    plt.close(figure)
    buf.seek(0)
    # 將記憶體中的資料轉成TensorFlow格式
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    image = tf.expand_dims(image, 0)
    return image

In [ ]:
# Example
y_true = [2, 0, 2, 2, 0, 1]
y_pred = [0, 0, 2, 2, 0, 2]
cm = tf.math.confusion_matrix(y_true, y_pred, num_classes=3).numpy()
img = plot_confusion_matrix(cm, [0, 1, 2])
img_show = plot_to_image(img)

### 創建Callback函數
創建回調函數：訓練過程中每個epoch結束，會產生一張Confusion matrix的圖片，並將圖片紀錄在Tensorboard上。

In [ ]:
class ConfusionMatrix(tf.keras.callbacks.Callback):
    def __init__(self, log_dir, test_data, class_name):
        super(ConfusionMatrix, self).__init__()
        self.log_dir = log_dir
        self.test_data = test_data
        self.class_names = class_name
        self.num_classes = len(class_name)

    def on_train_begin(self, logs=None):
        path = os.path.join(self.log_dir, 'confusion_matrix')
        # 創建TensorBoard紀錄檔
        self.writer = tf.summary.create_file_writer(path)

    def on_epoch_end(self, epoch, logs=None):
        # 計算Confusion matrix
        total_cm = np.zeros([10, 10])
        for x, y_true in self.test_data:
            y_pred = self.model.predict(x)
            y_pred = np.argmax(y_pred, axis=1)
            y_true = np.argmax(y_true, axis=1)
            cm = tf.math.confusion_matrix(y_true, y_pred, num_classes=self.num_classes).numpy()
            total_cm += cm
        
        # 將Confusion matrix轉成Matplotlib圖片
        figure = plot_confusion_matrix(total_cm, class_names=self.class_names)
        # 將Matplotlib圖片轉成TensorFlow型式的圖片
        cm_image = plot_to_image(figure)

        # 將圖片紀錄在TensorBoard log中
        with self.writer.as_default():
            tf.summary.image("Confusion Matrix", cm_image, step=epoch)

### 訓練網路模型

載入CIFAR-10數據集：

In [ ]:
# 將train Data重新分成9:1等分，分別分給train data, valid data
train_split, valid_split = ['train[:90%]', 'train[90%:]']
# 取得訓練數據，並順便讀取data的資訊
train_data, info = tfds.load("cifar10", split=train_split, with_info=True)
# 取得驗證數據
valid_data = tfds.load("cifar10", split=valid_split)
# 取得測試數據
test_data = tfds.load("cifar10", split=tfds.Split.TEST)
# 取得CIFAR-10數據集的類別
class_name = info.features['label'].names

Dataset 設定；

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE  # 自動調整模式
batch_size = 64  # 批次大小
train_num = int(info.splits['train'].num_examples / 10) * 9  # 訓練資料數量

train_data = train_data.shuffle(train_num)  # 打散資料集
# 載入預處理「 parse_aug_fn」function，cpu數量為自動調整模式
train_data = train_data.map(map_func=parse_aug_fn, num_parallel_calls=AUTOTUNE)
# 設定批次大小並將prefetch模式開啟(暫存空間為自動調整模式)
train_data = train_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

# 載入預處理「 parse_fn」function，cpu數量為自動調整模式
valid_data = valid_data.map(map_func=parse_fn, num_parallel_calls=AUTOTUNE)
# 設定批次大小並將prefetch模式開啟(暫存空間為自動調整模式)
valid_data = valid_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

# 載入預處理「 parse_fn」function，cpu數量為自動調整模式
test_data = test_data.map(map_func=parse_fn, num_parallel_calls=AUTOTUNE)
# 設定批次大小並將prefetch模式開啟(暫存空間為自動調整模式)
test_data = test_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

建立網路模型：

In [ ]:
inputs = keras.Input(shape=(32, 32, 3))
x = layers.Conv2D(64, 3, activation='relu', kernel_initializer='glorot_uniform')(inputs)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(128, 3, activation='relu', kernel_initializer='glorot_uniform')(x)
x = layers.Conv2D(256, 3, activation='relu', kernel_initializer='glorot_uniform')(x)
x = layers.Conv2D(128, 3, activation='relu', kernel_initializer='glorot_uniform')(x)
x = layers.Conv2D(64, 3, activation='relu', kernel_initializer='glorot_uniform')(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10)(x)
# 建立網路模型(將輸入到輸出所有經過的網路層連接起來)
model_1 = keras.Model(inputs, outputs, name='model-1')
model_1.summary()

建立Callback function：

In [ ]:
# 儲存訓練記錄檔
logs_dirs = 'lab7-logs-images'
model_cbk = keras.callbacks.TensorBoard(logs_dirs)
# 儲存Confusion matrix圖片
save_cm = ConfusionMatrix(logs_dirs, test_data, class_name)

設定訓練使用的優化器、客自化損失函數和客自化指標函數：

In [ ]:
model_1.compile(keras.optimizers.Adam(), 
                loss=keras.losses.CategoricalCrossentropy(from_logits=True), 
                metrics=[keras.metrics.CategoricalAccuracy()])

訓練網路模型：

In [ ]:
model_1.fit(train_data,
            epochs=100, 
            validation_data=valid_data,
            callbacks=[model_cbk, save_cm])

In [ ]:
%load_ext tensorboard

# 實驗二：使用TensorBoard超參數調校工具來訓練多個網路模型

In [ ]:
tensorboard --logdir lab7-logs-hparams/

### Import必要套件：

In [ ]:
import os
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers
# 從資料夾中的preprocessing.py檔案中Import parse_aug_fn和parse_fn函數
from preprocessing import parse_aug_fn, parse_fn

### Import TensorBoard超參數工具所需要的套件

In [ ]:
from tensorboard.plugins.hparams import api as hp

### 設定TensorBoard超參數調校

In [ ]:
hparam_ia = hp.HParam('Imgae_Augmentation', hp.Discrete([False, True]))
hparam_bn = hp.HParam('Batch_Normalization', hp.Discrete([False, True]))
hparam_init = hp.HParam('Weight_Initialization', hp.Discrete(['RandomNormal_0.01std', 'glorot_normal', 'he_normal']))
hparam_lr = hp.HParam('Learning_Rate', hp.Discrete([0.001, 0.01, 0.03]))

### 將實驗摘要寫入紀錄檔

In [ ]:
metric = 'Accuracy'
log_dirs = "lab7-logs-hparams/hparam_tuning"
with tf.summary.create_file_writer(log_dirs).as_default():
    hp.hparams_config(
        hparams=[hparam_ia, hparam_bn, hparam_init, hparam_lr],
        metrics=[hp.Metric(metric, display_name='Accuracy')],
    )

In [ ]:
%pwd

In [ ]:
%load_ext tensorboard

In [ ]:
tensorboard --logdir lab7-logs-hparams/

### 訓練網路模型

準備訓練資料，分別為「沒有影像增強」和「有影像增強」的訓練資料

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE  # 自動調整模式
batch_size = 64  # 批次大小
# 將train Data重新分成1:9等分，分別分給valid data, train data
train_split, valid_split = ['train[:90%]', 'train[90%:]']

# 取得訓練數據
train_data_noaug, info = tfds.load("cifar10", split=train_split, with_info=True)
train_data_aug = tfds.load("cifar10", split=train_split)
# 取得驗證數據
valid_data = tfds.load("cifar10", split=valid_split)

train_num = int(info.splits['train'].num_examples / 5) * 4  # 訓練資料數量

train_data_noaug = train_data_noaug.shuffle(train_num)  # 打散資料集
# 載入預處理「 parse_aug_fn」function，cpu數量為自動調整模式
train_data_noaug = train_data_noaug.map(map_func=parse_fn, num_parallel_calls=AUTOTUNE)
# 設定批次大小並將prefetch模式開啟(暫存空間為自動調整模式)
train_data_noaug = train_data_noaug.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

train_data_aug = train_data_aug.shuffle(train_num)  # 打散資料集
# 載入預處理「 parse_fn」function，cpu數量為自動調整模式
train_data_aug = train_data_aug.map(map_func=parse_aug_fn, num_parallel_calls=AUTOTUNE)
# 設定批次大小並將prefetch模式開啟(暫存空間為自動調整模式)
train_data_aug = train_data_aug.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

# 載入預處理「 parse_fn」function，cpu數量為自動調整模式
valid_data = valid_data.map(map_func=parse_fn, num_parallel_calls=AUTOTUNE)
# 設定批次大小並將prefetch模式開啟(暫存空間為自動調整模式)
valid_data = valid_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

客製化超參數回調函式：用於紀錄每一次訓練模型的超參數數值和最終訓練結果的指標值。

In [ ]:
class HyperparameterCallback(tf.keras.callbacks.Callback):
    # 類別創建時調用
    def __init__(self, log_dir, hparams):
        super(HyperparameterCallback, self).__init__()
        self.log_dir = log_dir
        self.hparams = hparams
        self.best_accuracy = 0
        self.writer = None
        
    # 訓練開始前調用
    def on_train_begin(self, logs=None):
        self.writer = tf.summary.create_file_writer(self.log_dir)

    # 每一個Epcoh結束後調用
    def on_epoch_end(self, epoch, logs=None):
        current_accuracy = logs.get('val_categorical_accuracy')
        if current_accuracy > self.best_accuracy:
            self.best_accuracy = current_accuracy
            
    # 訓練結束時調用
    def on_train_end(self, logs=None):
        with self.writer.as_default():
            hp.hparams(self.hparams)  # record the values used in this trial
            tf.summary.scalar(metric, self.best_accuracy, step=0)

建立一個函式負責創建、編譯和訓練網路模型，網路層配置如下：
- keras.Input：輸入層(輸入影像大小為32x32x3)。
- layers.Conv2D：卷積層(使用ReLU激活函數，以及3x3大小的kernel)。
- layers.MaxPool2D：池化層(對特徵圖下採樣)。
- layers.Flatten：扁平層(特徵圖轉成一維Tensor)。
- layers.Dropout：Dropout層(每次訓練隨機丟棄50%網路)。
- layers.Dense：全連接層(隱藏層使用ReLU激活函數，輸出層使用Softmax激活函數)。

In [ ]:
def train_test_model(logs_dir, hparams):
    """
    logs_dir:傳入目前執行的任務log檔的位置
    hparams:傳入超參數
    """
    # 指派網路模型初始化的方法
    if hparams[hparam_init] == "glorot_normal":
        init = keras.initializers.glorot_normal()
    elif hparams[hparam_init] == "he_normal":
        init = keras.initializers.he_normal()
    else:
        init = keras.initializers.RandomNormal(0, 0.01)

    inputs = keras.Input(shape=(32, 32, 3))
    x = layers.Conv2D(64, (3, 3), kernel_initializer=init)(inputs)
    if hparams[hparam_bn]: x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPool2D()(x)
    x = layers.Conv2D(128, (3, 3), kernel_initializer=init)(x)
    if hparams[hparam_bn]: x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(256, (3, 3), kernel_initializer=init)(x)
    if hparams[hparam_bn]: x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(128, (3, 3), kernel_initializer=init)(x)
    if hparams[hparam_bn]: x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(64, (3, 3), kernel_initializer=init)(x)
    if hparams[hparam_bn]: x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, kernel_initializer=init)(x)
    if hparams[hparam_bn]: x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(10, activation='softmax')(x)
    # 建立網路模型(將輸入到輸出所有經過的網路層連接起來)
    model = keras.Model(inputs, outputs, name='model')

    # 儲存訓練記錄檔
    model_tb = keras.callbacks.TensorBoard(log_dir=logs_dir, write_graph=False)

    # 儲存最好的網路模型權重
    model_mckp = keras.callbacks.ModelCheckpoint(logs_dir +'/best-model.hdf5', 
                                                 monitor='val_categorical_accuracy', 
                                                 save_best_only=True, 
                                                 mode='max')
    
    # 設定停止訓練的條件(當Accuracy超過30迭代沒有上升的話訓練會終止)
    model_els = keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy' , 
                                              min_delta=0, 
                                              patience=30, 
                                              mode='max')
    # 客自化超參數回調函式，紀錄訓練模型的超參數和指標(準確率)
    model_hparam = HyperparameterCallback(logs_dir + 'hparam_tuning', hparams)


    # 設定訓練使用的優化器、損失函數和指標函數
    # 優化器學習率為超參數：0.001、0.01或0.03
    model.compile(keras.optimizers.Adam(hparams[hparam_lr]), 
                  loss=keras.losses.CategoricalCrossentropy(), 
                  metrics=[keras.metrics.CategoricalAccuracy()])
    
    # 超參數：使用「經過影像增強的數據」或「不經過影像增強的數據」訓練網路
    if hparams[hparam_ia]:
        history = model.fit(train_data_aug,
                            epochs=2, 
                            validation_data=valid_data,
                            callbacks=[model_tb, model_mckp, model_els, model_hparam])
    else:
        history = model.fit(train_data_noaug,
                            epochs=2, 
                            validation_data=valid_data,
                            callbacks=[model_tb, model_mckp, model_els, model_hparam])

訓練36種不同參數的網路模型：

In [ ]:
session_id = 1  # 訓練任務的id
for ia in hparam_ia.domain.values:
    for bn in hparam_bn.domain.values:
        for init in hparam_init.domain.values:
            for lr in hparam_lr.domain.values:
                # 顯示目前訓練任務id
                print('--- Running training session {}'.format(session_id))
                # 設定本次訓練的超參數
                hparams = {hparam_ia: ia, hparam_bn: bn, hparam_init: init, hparam_lr: lr}
                # 儲放紀錄檔的位置
                logs_dir = os.path.join("lab7-logs-hparams", "run-{}".format(session_id))
                # 建立、編譯及訓練網路模型
                train_test_model(logs_dir, hparams)
                session_id += 1  # id+1